<a href="https://colab.research.google.com/github/Chahethsen12/The_Data_Detective/blob/main/The_Data_Detective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [ ]:
!pip install -q gradio scikit-learn pandas
print("Ready to go!")

**Load Data & Train Model** *We will use a direct link to the Telco Churn dataset so you don't have to upload files manually.*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# 1. Load Data directly from GitHub
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)

# 2. Quick Cleaning
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)
df = df.dropna()

# Encode text to numbers
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# 3. Train Model
X = df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Contract', 'PaymentMethod']] # Selecting top features
y = df['Churn']

model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)
print("Model trained! Accuracy score:", model.score(X, y))

The UI (Gradio)

In [ ]:
import gradio as gr
import numpy as np

def predict_churn(tenure, monthly_charges, total_charges, contract, payment_method):
    # Prepare input (must match training order)
    # Note: For a real app, we'd map text inputs back to the LabelEncoder numbers.
    # Here we are simplifying for the demo.
    input_data = np.array([[tenure, monthly_charges, total_charges, contract, payment_method]])
    prediction = model.predict(input_data)
    probability = model.predict_proba(input_data)[0][1]

    result = "🔴 Likely to Churn" if prediction[0] == 1 else "🟢 Likely to Stay"
    return f"{result} (Probability: {probability:.2%})"

# Create the Interface
interface = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Slider(0, 72, label="Tenure (Months)"),
        gr.Slider(10, 200, label="Monthly Charges ($)"),
        gr.Number(label="Total Charges ($)"),
        gr.Slider(0, 2, step=1, label="Contract Type (0=Month, 1=1yr, 2=2yr)"),
        gr.Slider(0, 3, step=1, label="Payment Method (0-3 code)")
    ],
    outputs="text",
    title="Customer Churn Predictor",
    description="Adjust sliders to see if the AI predicts the customer will leave."
)

interface.launch()